# Crée des images à partir des images existantes

In [1]:
# !pip install import-ipynb

In [2]:
import import_ipynb

In [3]:
%run iteration_0_imports.py

In [4]:
%run iteration_0_parameters.py

In [5]:
import iteration_0_utils as utils

importing Jupyter notebook from iteration_0_utils.ipynb


In [6]:
%whos

Variable                          Type        Data/Info
-------------------------------------------------------
HEALTHY                           str         H
IMG_IN_COLOR                      int         1
IMG_SIZE                          int         64
Image                             module      <module 'PIL.Image' from <...>packages\\PIL\\Image.py'>
NBR_SAMPLE                        int         20000
PATH_DATA                         str         ../data
PATH_DATA_EDA                     str         ../data/eda
PATH_DATA_LIST                    str         ../data/list
PATH_DATA_SAMPLE                  str         ../data/sample
PATH_MODEL                        str         ../model
PATH_ROOT                         str         ..
PATH_SOURCE_IMG                   str         ../cell_images
RandomForestClassifier            ABCMeta     <class 'sklearn.ensemble.<...>.RandomForestClassifier'>
SICK                              str         S
classification_report             function

In [7]:
df_path_img_sick = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_sick_img.csv")
df_path_img_healthy = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_healthy_img.csv")
df_path_img = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_img.csv")

In [8]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [9]:
df_path_img_sick.head()

,path_img
0,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_162.png
1,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_163.png
2,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_164.png
3,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_165.png
4,../cell_images/Parasitized\C100P61ThinF_IMG_20150918_144104_cell_166.png


In [11]:
NBR_SAMPLE = 10

Flip de l'image : https://note.nkmk.me/en/python-opencv-numpy-rotate-flip/

In [12]:
FLIP_UP_DOWN = 0
FLIP_LEFT_RIGHT = 1
FLIP_DIAGONAL = -1
# cv2.flip(img_arr, FLIP_UP_DOWN)

Suppression du padding autours de limage : https://codereview.stackexchange.com/questions/132914/crop-black-border-of-image-using-numpy

In [13]:
def crop_img(img_arr: np.array) -> np.array:
    """
    Supprime le padding autours de l'image pour une dimension minimale
    """
    
    img_arr_sum = img_arr.sum(axis=2) #Traite les 3 couleurs comme une seule.
    # Mask of non-black pixels (assuming image has a single channel).
    mask = img_arr_sum > 0

    # Coordinates of non-black pixels.
    coords = np.argwhere(mask)

    # Bounding box of non-black pixels.
    x0, y0 = coords.min(axis=0)
    x1, y1 = coords.max(axis=0) + 1   # slices are exclusive at the top
    
    # Get the contents of the bounding box.
    img_arr_cropped = img_arr[x0:x1, y0:y1]
    return img_arr_cropped

Rotation de l'image : https://stackoverflow.com/questions/43892506/opencv-python-rotate-image-without-cropping-sides

In [14]:
def rotate_image(img_arr: np.array, angle: float) -> np.array:
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    """

    height, width = img_arr.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_img_arr = cv2.getRotationMatrix2D(image_center, -angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_img_arr[0,0]) 
    abs_sin = abs(rotation_img_arr[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_img_arr[0, 2] += bound_w/2 - image_center[0]
    rotation_img_arr[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_img_arr = cv2.warpAffine(img_arr, rotation_img_arr, (bound_w, bound_h))
    return rotated_img_arr

Stratégie de création d'image
 * Décider si on fait tourner l'image ou si on la flip
 * Si on la fait tourner, décider combien de rotation et de l'angle de chacune de ces rotations.
 * Si on la flip, décider quel flip appliquer.

In [15]:
from numpy.random import default_rng

In [34]:
def create_random_img_old(path_img: str, result_root_path: str = PATH_SOURCE_IMG):
    """
    Créer des images dans le répertoire résultat
    """
    file_path, file_name, file_extention = utils.split_file_info(path_img)
    img_arr = cv2.imread(path_img, flags = IMG_IN_COLOR)
    file_class = file_path.split("/")[-1]
    
    #print(file_class)
    
    PROCESS_FLIP = 0
    PROCESS_ROTATION = 1
    
    process_choice = np.random.randint(0, 2)
    
    if process_choice == PROCESS_FLIP:
        #print("flip")
        #Flip
        flip_choice = np.random.choice([FLIP_UP_DOWN, FLIP_LEFT_RIGHT, FLIP_DIAGONAL])
        img_flipped = cv2.flip(img_arr, flip_choice)
        
        img_result_file_info = result_root_path + "/Generated_" + file_class + "/" + \
                               file_name + "_flip_" + str(flip_choice) + "." + file_extention
        #print(img_result_file_info)
        
        cv2.imwrite(img_result_file_info, img_flipped)
        
    elif process_choice == PROCESS_ROTATION:
        #print("rot")
        #Rotation
        rotation_number = np.random.randint(1, 10 + 1)
        rng = default_rng()
        angle_list = rng.choice(359, size=rotation_number, replace=False) + 1 #Angles entre 1 et 359
        
        for angle in angle_list:
            #print(angle)
            img_arr_rotated = rotate_image(img_arr, angle)
            img_arr_rotated = crop_img(img_arr_rotated)
            img_result_file_info = result_root_path + "/Generated_" + file_class + "/" + \
                                   file_name + "_rot_" + str(angle) + "." + file_extention
            #print(img_result_file_info)
            
            cv2.imwrite(img_result_file_info, img_arr_rotated)
            
    else:
        assert False
    

In [59]:
def ramdom_img_creation_policy():
    process_choice = np.random.randint(0, 2)
    
    flip_choice = np.random.choice([FLIP_UP_DOWN, FLIP_LEFT_RIGHT, FLIP_DIAGONAL])
    
    rotation_number = np.random.randint(1, 10 + 1)
    rng = default_rng()
    angle_list = rng.choice(359, size=rotation_number, replace=False) + 1 #Angles entre 1 et 359
    
    return process_choice, flip_choice, angle_list

In [60]:
def create_random_img(path_img: str, result_root_path: str, \
                      process_choice: int, \
                      flip_choice: int, \
                      angle_list: np.array):
    """
    Créer des images dans le répertoire résultat
    """
    file_path, file_name, file_extention = utils.split_file_info(path_img)
    img_arr = cv2.imread(path_img, flags = IMG_IN_COLOR)
    file_class = file_path.split("/")[-1]
    
    #print(file_class)
    
    PROCESS_FLIP = 0
    PROCESS_ROTATION = 1
    
    if process_choice == PROCESS_FLIP:
        #print("flip")
        #Flip
        img_flipped = cv2.flip(img_arr, flip_choice)
        
        img_result_file_info = result_root_path + "/Generated_" + file_class + "/" + \
                               file_name + "_flip_" + str(flip_choice) + "." + file_extention
        #print(img_result_file_info)
        
        cv2.imwrite(img_result_file_info, img_flipped)
        
    elif process_choice == PROCESS_ROTATION:
        #print("rot")
        #Rotation
        for angle in angle_list:
            #print(angle)
            img_arr_rotated = rotate_image(img_arr, angle)
            img_arr_rotated = crop_img(img_arr_rotated)
            img_result_file_info = result_root_path + "/Generated_" + file_class + "/" + \
                                   file_name + "_rot_" + str(angle) + "." + file_extention
            #print(img_result_file_info)
            
            cv2.imwrite(img_result_file_info, img_arr_rotated)
            
    else:
        assert False

        

In [63]:
NBR_GENERATED = 1000

for img_sick_path, img_healty_path in tqdm(zip(df_path_img_sick.sample(NBR_GENERATED).path_img, \
                                          df_path_img_healthy.sample(NBR_GENERATED).path_img)):
    process_choice, flip_choice, angle_list = ramdom_img_creation_policy()
    
    create_random_img(img_sick_path, PATH_SOURCE_IMG, \
                      process_choice, flip_choice, angle_list)
    
    create_random_img(img_healty_path, PATH_SOURCE_IMG, \
                  process_choice, flip_choice, angle_list)

1000it [01:06, 14.96it/s]
